
# GPU Numba and CuPy Parallelization of Matrix Multiplication 

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.

In [1]:
import numpy as np
import numba

<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix. 

**Before you start**, take a look at the following:
* Read over the following notebook that explanes Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb) 
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [2]:
from numba import cuda

@cuda.jit
def transpose_kernel(A, B):
    i, j = cuda.grid(2)
    N = A.shape[0]
    if i < N and j < N:
        B[i, j] = A[j, i]

In [3]:
# Input Matrix
N = 1024
A = np.random.rand(N, N)

# Output will be the same size as the input matrix
B = np.zeros((N, N))

# Define the block size and grid size
block_size = (32, 32)
grid_size = (int(np.ceil(N/block_size[0])), int(np.ceil(N/block_size[1])))

# Kernel
transpose_kernel[grid_size, block_size](A, B)

/home/spack/spack/opt/spack/linux-centos7-haswell/gcc-9.5.0/python-3.10.7-xj7xqdemt2mshnxfntrc7bq6nyx6khcv/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication: 

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package. 

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function. 
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."
    
<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```    
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)
    
<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime." 

In [71]:
# defining a function decorator for timing other functions

from functools import wraps
from time import time, sleep
from itertools import permutations


def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):4.3f}s\n')
        return result
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

In [72]:
Ntest=32
N1=5120
N2=10240
N3=20480

#I individually changed size of N to look at the role that matrix size plays.
#I wasn't sure/too lazy to find an efficient way to loop :3

In [73]:
#h_A = np.random.random((N1, N1)).astype(np.float32)
#h_B = np.random.random((N1, N1)).astype(np.float32)
h_A = np.random.random((N1, N1))
h_B = np.random.random((N1, N1))

h_A32 = h_A.astype(np.float32)
h_B32 = h_A.astype(np.float32)

h_A64 = h_A.astype(np.float64)
h_B64 = h_A.astype(np.float64)

In [74]:
from __future__ import division
from numba import cuda
import numpy
import math

#Initialize Total Event
start_event = cuda.event()
end_event = cuda.event()

# Record the Start of Total event
start_event.record()


# CUDA kernel
numba.cuda.event(timing=True)
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

# Initialize the data arrays
#A = numpy.full((24, 12), 3, numpy.float) # matrix containing all 3's
#B = numpy.full((12, 22), 4, numpy.float) # matrix containing all 4's


#Initialize Copy Event
start_copyevent = cuda.event()
end_copyevent = cuda.event()

# Record the Start of Total event
start_copyevent.record()

# Copy the arrays to the device -- the equivalent of d_A and d_B
A_global_mem = cuda.to_device(h_A32)
B_global_mem = cuda.to_device(h_B32)

# Record the end event
end_copyevent.record()

# Synchronize to wait for the events to complete
end_copyevent.synchronize()

# Compute the Copy elapsed time
elapsed_copytime = cuda.event_elapsed_time(start_copyevent, end_copyevent)
print(elapsed_copytime/1000)


# Allocate memory on the device for the result -- the equivalent of d_C
C_global_mem = cuda.device_array((h_A.shape[0],h_B.shape[1]))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(h_A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(h_B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host -- the equivalent of h_C
C32 = C_global_mem.copy_to_host()


print(C32)

# Record the end event
end_event.record()

# Synchronize to wait for the events to complete
end_event.synchronize()

# Compute the total elapsed time
elapsed_time = cuda.event_elapsed_time(start_event, end_event)
print(elapsed_time/1000)

0.10200662231445312
[[1265.58932836 1268.49147887 1276.416139   ... 1278.68302937
  1249.71170104 1294.96838136]
 [1274.93335371 1258.11072075 1269.63152314 ... 1269.13949979
  1253.3997985  1295.87762923]
 [1296.0086385  1281.10968116 1282.40763949 ... 1285.24985428
  1269.15932237 1298.0536378 ]
 ...
 [1294.25286303 1280.99521044 1287.78793514 ... 1284.76990299
  1268.47559971 1317.8841791 ]
 [1299.75554432 1307.66221258 1304.37306205 ... 1306.63218167
  1279.94072433 1330.59064604]
 [1293.12331209 1275.12425777 1286.82023674 ... 1270.10158207
  1259.01523927 1303.02700646]]
3.33952099609375


In [75]:
from __future__ import division
from numba import cuda
import numpy
import math

#Initialize Total Event
start_event = cuda.event()
end_event = cuda.event()

# Record the Start of Total event
start_event.record()


# CUDA kernel
numba.cuda.event(timing=True)
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

# Initialize the data arrays
#A = numpy.full((24, 12), 3, numpy.float) # matrix containing all 3's
#B = numpy.full((12, 22), 4, numpy.float) # matrix containing all 4's

#Initialize Copy Event
start_copyevent = cuda.event()
end_copyevent = cuda.event()

# Record the Start of Total event
start_copyevent.record()

# Copy the arrays to the device -- the equivalent of d_A and d_B
A_global_mem = cuda.to_device(h_A64)
B_global_mem = cuda.to_device(h_B64)

# Record the end event
end_copyevent.record()

# Synchronize to wait for the events to complete
end_copyevent.synchronize()

# Compute the Copy elapsed time
elapsed_copytime = cuda.event_elapsed_time(start_copyevent, end_copyevent)
print(elapsed_copytime/1000)


# Allocate memory on the device for the result -- the equivalent of d_C
C_global_mem = cuda.device_array((h_A.shape[0],h_B.shape[1]))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(h_A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(h_B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host -- the equivalent of h_C
C64 = C_global_mem.copy_to_host()


print(C64)

# Record the end event
end_event.record()

# Synchronize to wait for the events to complete
end_event.synchronize()

# Compute the total elapsed time
elapsed_time = cuda.event_elapsed_time(start_event, end_event)
print(elapsed_time/1000)

0.09427375793457031
[[1265.589327   1268.49147801 1276.41613835 ... 1278.68302876
  1249.71170082 1294.96837963]
 [1274.93335264 1258.11072159 1269.63152412 ... 1269.13949882
  1253.39979946 1295.87762961]
 [1296.00863823 1281.10968211 1282.4076388  ... 1285.24985331
  1269.15932145 1298.05363662]
 ...
 [1294.25286023 1280.99521188 1287.78793433 ... 1284.7699011
  1268.47559898 1317.88417874]
 [1299.755544   1307.66221104 1304.37306188 ... 1306.63217996
  1279.94072431 1330.59064521]
 [1293.12331176 1275.12425898 1286.82023578 ... 1270.10157992
  1259.01524025 1303.02700638]]
5.128408203125


In [82]:
#Comparing against numpy.dot
@mytimer
def npcomparison(A,B):
    C=np.dot(A,B)
    return C
C1=npcomparison(h_A32,h_B32)
print(C1)

Function: 'npcomparison({})' executed in 0.575s

[[1265.5894 1268.4912 1276.4161 ... 1278.6832 1249.7117 1294.9683]
 [1274.9335 1258.1107 1269.6315 ... 1269.1395 1253.3999 1295.8777]
 [1296.0088 1281.1096 1282.4076 ... 1285.2499 1269.1593 1298.0537]
 ...
 [1294.2529 1280.9951 1287.7881 ... 1284.7701 1268.4755 1317.884 ]
 [1299.7557 1307.6624 1304.3733 ... 1306.6322 1279.9406 1330.5906]
 [1293.1234 1275.1243 1286.8202 ... 1270.1017 1259.0154 1303.027 ]]


In [83]:
Cdiff32=C1-C32
Cdiff64=C1-C64
print(Cdiff32)
print(Cdiff64)

[[ 2.71126628e-05 -2.67931962e-04 -1.30108447e-06 ...  1.98164433e-04
  -3.11157910e-05 -1.19640388e-04]
 [ 1.17973380e-04 -2.97489896e-06 -5.34141241e-05 ...  2.65808103e-05
   1.03847340e-04  5.63160138e-05]
 [ 1.50564225e-04 -6.20173669e-05 -4.67116840e-05 ...  2.36518172e-05
  -2.06082641e-05  7.31368759e-05]
 ...
 [ 6.66563028e-05 -9.32480798e-05  1.50794437e-04 ...  2.38613055e-04
  -1.35846298e-04 -1.45897433e-04]
 [ 1.92986185e-04  1.40938556e-04  2.28970177e-04 ...  2.04776763e-05
  -1.72568635e-04 -6.98667509e-05]
 [ 1.00993620e-04  9.81204721e-06 -4.63147221e-05 ...  1.02495454e-04
   1.41592253e-04 -2.89198833e-05]]
[[ 2.84702196e-05 -2.67073501e-04 -6.54315272e-07 ...  1.98783464e-04
  -3.08978370e-05 -1.17911885e-04]
 [ 1.19038325e-04 -3.81651762e-06 -5.43970568e-05 ...  2.75463340e-05
   1.02885172e-04  5.59336117e-05]
 [ 1.50828404e-04 -6.29690630e-05 -4.60249139e-05 ...  2.46187778e-05
  -1.96879394e-05  7.43182436e-05]
 ...
 [ 6.94617954e-05 -9.46883645e-05  1.5161224

In [84]:
from __future__ import division
from numba import cuda, float32
import numpy
import math

start_event = cuda.event()
end_event = cuda.event()

# Record the start event
start_event.record()

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
#A = numpy.full((TPB*2, TPB*3), 3, numpy.float32) # [32 x 48] matrix containing all 3's
#B = numpy.full((TPB*3, TPB*1), 4, numpy.float32) # [48 x 16] matrix containing all 4's


# Record the Start of Total event
start_event.record()

#Initialize Copy Event
start_copyevent = cuda.event()
end_copyevent = cuda.event()

# Record the Start of Copy event
start_copyevent.record()

A_global_mem = cuda.to_device(h_A32)
B_global_mem = cuda.to_device(h_B32)
C_global_mem = cuda.device_array((h_A.shape[0],h_B.shape[1]))

# Record the end event
end_copyevent.record()

# Synchronize to wait for the events to complete
end_copyevent.synchronize()

# Compute the Copy elapsed time
elapsed_copytime = cuda.event_elapsed_time(start_copyevent, end_copyevent)
print(elapsed_copytime/1000)

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(h_A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(h_B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res32 = C_global_mem.copy_to_host()

print(res)

# Record the end event
end_event.record()

# Synchronize to wait for the events to complete
end_event.synchronize()

# Compute the elapsed time
elapsed_time = cuda.event_elapsed_time(start_event, end_event)
print(elapsed_time/1000)

0.05046992111206055
[[1265.58932836 1268.49147887 1276.416139   ... 1278.68302937
  1249.71170104 1294.96838136]
 [1274.93335371 1258.11072075 1269.63152314 ... 1269.13949979
  1253.3997985  1295.87762923]
 [1296.0086385  1281.10968116 1282.40763949 ... 1285.24985428
  1269.15932237 1298.0536378 ]
 ...
 [1294.25286303 1280.99521044 1287.78793514 ... 1284.76990299
  1268.47559971 1317.8841791 ]
 [1299.75554432 1307.66221258 1304.37306205 ... 1306.63218167
  1279.94072433 1330.59064604]
 [1293.12331209 1275.12425777 1286.82023674 ... 1270.10158207
  1259.01523927 1303.02700646]]
3.042510986328125


In [85]:
from __future__ import division
from numba import cuda, float32
import numpy
import math

start_event = cuda.event()
end_event = cuda.event()

# Record the start event
start_event.record()

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
#A = numpy.full((TPB*2, TPB*3), 3, numpy.float32) # [32 x 48] matrix containing all 3's
#B = numpy.full((TPB*3, TPB*1), 4, numpy.float32) # [48 x 16] matrix containing all 4's


# Record the Start of Total event
start_event.record()

#Initialize Copy Event
start_copyevent = cuda.event()
end_copyevent = cuda.event()

# Record the Start of Copy event
start_copyevent.record()

A_global_mem = cuda.to_device(h_A64)
B_global_mem = cuda.to_device(h_B64)
C_global_mem = cuda.device_array((h_A.shape[0],h_B.shape[1]))

# Record the end event
end_copyevent.record()

# Synchronize to wait for the events to complete
end_copyevent.synchronize()

# Compute the Copy elapsed time
elapsed_copytime = cuda.event_elapsed_time(start_copyevent, end_copyevent)
print(elapsed_copytime/1000)

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(h_A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(h_B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res64 = C_global_mem.copy_to_host()

print(res)

# Record the end event
end_event.record()

# Synchronize to wait for the events to complete
end_event.synchronize()

# Compute the elapsed time
elapsed_time = cuda.event_elapsed_time(start_event, end_event)
print(elapsed_time/1000)

0.09278489685058594
[[1265.58932836 1268.49147887 1276.416139   ... 1278.68302937
  1249.71170104 1294.96838136]
 [1274.93335371 1258.11072075 1269.63152314 ... 1269.13949979
  1253.3997985  1295.87762923]
 [1296.0086385  1281.10968116 1282.40763949 ... 1285.24985428
  1269.15932237 1298.0536378 ]
 ...
 [1294.25286303 1280.99521044 1287.78793514 ... 1284.76990299
  1268.47559971 1317.8841791 ]
 [1299.75554432 1307.66221258 1304.37306205 ... 1306.63218167
  1279.94072433 1330.59064604]
 [1293.12331209 1275.12425777 1286.82023674 ... 1270.10158207
  1259.01523927 1303.02700646]]
3.29268310546875


In [86]:
#Comparing against numpy.dot
#Comparing against numpy.dot
#Comparing against numpy.dot
@mytimer
def npcomparison(A,B):
    C=np.dot(A,B)
    return C
C2=npcomparison(h_A32,h_B32)
print(C2)

Function: 'npcomparison({})' executed in 0.550s

[[1265.5894 1268.4912 1276.4161 ... 1278.6832 1249.7117 1294.9683]
 [1274.9335 1258.1107 1269.6315 ... 1269.1395 1253.3999 1295.8777]
 [1296.0088 1281.1096 1282.4076 ... 1285.2499 1269.1593 1298.0537]
 ...
 [1294.2529 1280.9951 1287.7881 ... 1284.7701 1268.4755 1317.884 ]
 [1299.7557 1307.6624 1304.3733 ... 1306.6322 1279.9406 1330.5906]
 [1293.1234 1275.1243 1286.8202 ... 1270.1017 1259.0154 1303.027 ]]


In [87]:
C2diff=C2-res
print(C2diff)

C2diff32=C2-res32
C2diff64=C2-res64
print(C2diff32)
print(C2diff64)

[[ 2.71126628e-05 -2.67931962e-04 -1.30108447e-06 ...  1.98164433e-04
  -3.11157910e-05 -1.19640388e-04]
 [ 1.17973380e-04 -2.97489896e-06 -5.34141241e-05 ...  2.65808103e-05
   1.03847340e-04  5.63160138e-05]
 [ 1.50564225e-04 -6.20173669e-05 -4.67116840e-05 ...  2.36518172e-05
  -2.06082641e-05  7.31368759e-05]
 ...
 [ 6.66563028e-05 -9.32480798e-05  1.50794437e-04 ...  2.38613055e-04
  -1.35846298e-04 -1.45897433e-04]
 [ 1.92986185e-04  1.40938556e-04  2.28970177e-04 ...  2.04776763e-05
  -1.72568635e-04 -6.98667509e-05]
 [ 1.00993620e-04  9.81204721e-06 -4.63147221e-05 ...  1.02495454e-04
   1.41592253e-04 -2.89198833e-05]]
[[ 2.71126628e-05 -2.67931962e-04 -1.30108447e-06 ...  1.98164433e-04
  -3.11157910e-05 -1.19640388e-04]
 [ 1.17973380e-04 -2.97489896e-06 -5.34141241e-05 ...  2.65808103e-05
   1.03847340e-04  5.63160138e-05]
 [ 1.50564225e-04 -6.20173669e-05 -4.67116840e-05 ...  2.36518172e-05
  -2.06082641e-05  7.31368759e-05]
 ...
 [ 6.66563028e-05 -9.32480798e-05  1.5079443

<br>

### Exercise 2: Response

Both the numpy and cuda uses vastly increase the speedup for the matrix multiplacation compared to the naive CPU matrix version. For all of the matrix cases except for the largest one, numpy was faster than the GPUs. As the matrix sizes got bigger, it became more and more favorable to use the GPU's. The GPUs were twice as slow for float64 than float32. This effect isn't as stark as the change in speeds for increasing matrix sizes. The biggest time holdup was just waiting for the data to be copied to the GPU and then for the information to come back. Also, it was only worth using the GPU when they were all linked up (second copied NYU code) -- so knowing how to configure them. For the most part, it's easiest for the programmer to live in numpy world. Apologies for not looping throught all of the N's.

<br>

### Exercise 3: CuPy 

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [115]:
import numpy as np
import cupy as cp
import time

def test_cupy_dot32():
    h_A = np.random.random((N1,N1)).astype('float32') # generate a random matrix on the CPU
    h_B = np.random.random((N1,N1)).astype('float32')
    d_A = cp.asarray(h_A)
    d_B = cp.asarray(h_B)
    start_event = cp.cuda.Event()
    end_event = cp.cuda.Event()
    start_event.record()
    #CuPy
    d_C = cp.dot(d_A, d_B)
    end_event.record()
    end_event.synchronize()
    gpu_time = cp.cuda.get_elapsed_time(start_event, end_event)
    h_C = d_C.get()
    h_ref = np.dot(h_A, h_B)
    error = np.linalg.norm(h_ref - h_C) / np.linalg.norm(h_ref)
    print(gpu_time/100)
    print(error)

In [122]:
import numpy as np
import cupy as cp
import time

def test_cupy_dot64():
    h_A = np.random.random((N1,N1)).astype('float64') # generate a random matrix on the CPU
    h_B = np.random.random((N1,N1)).astype('float64')
    d_A = cp.asarray(h_A)
    d_B = cp.asarray(h_B)
    start_event = cp.cuda.Event()
    end_event = cp.cuda.Event()
    start_event.record()
    #CuPy
    d_C = cp.dot(d_A, d_B)
    end_event.record()
    end_event.synchronize()
    gpu_time = cp.cuda.get_elapsed_time(start_event, end_event)
    h_C = d_C.get()
    h_ref = np.dot(h_A, h_B)
    error = np.linalg.norm(h_ref - h_C) / np.linalg.norm(h_ref)
    print(gpu_time/100)
    print(error)

In [123]:
print('time to run and error for d32')
test_cupy_dot32()
print('time to run and error for d64')
test_cupy_dot64()

time to run and error for d32
0.9117759704589844
1.1077095e-06
time to run and error for d64
11.995714111328125
2.064249053257417e-15


<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

In [125]:
def test_matmul(d_A, d_B):
    # Wrap the device arrays
    wrapped_d_A = cp.asarray(d_A)
    wrapped_d_B = cp.asarray(d_B)

    # Calculate matrix multiplication using cupy.dot()
    start_wrap = time.time()
    result = cp.dot(wrapped_d_A, wrapped_d_B)
    end_wrap = time.time()

    # copy result back to host memory
    h_result = result.get()

    # compare with reference numpy result
    h_ref = np.dot(d_A.copy_to_host(), d_B.copy_to_host())
    error = np.linalg.norm(h_ref - h_result) / np.linalg.norm(h_ref)

    # Print results
    # print(f"Cupy dot result:\n{h_result}\n")
    # print(f"Reference numpy dot result:\n{h_ref}\n")
    print(error)
    print(f"Time to wrap device arrays: {end_wrap - start_wrap:.5f} seconds")
    print(f"Time to perform matrix multiplication: {end_wrap - start_wrap:.5f} seconds")
    
def test_wrap():
    d_A = cuda.to_device(h_A)
    d_B = cuda.to_device(h_B)

    #  test_matmul function to wrap device arrays and perform matrix multiplication
    test_matmul(d_A, d_B)
            
test_wrap()

2.0658392160139756e-15
Time to wrap device arrays: 0.00018 seconds
Time to perform matrix multiplication: 0.00018 seconds


<br>

### Exercise $\mathbf{\pi}$: Response

The runtimes are lowest for the test_wrap scenario. With excercise 3, we would have to pay the cost of wrapping for each iteration we ran, which adds up. For this case, we only have to pay the cost once because we wrap over all of the runs. It takes a lot of time for the information to copy over between machines. We want to minimize the copying and reading as much as possible.